BELOW CODE SECTION IS A ROUGH SECTION

IMPORTANT INSTRCUTIONS

1. Use as minimum variables as needed
2. try to reuse exsting variables.
3. Delete the variables if not used to save the RAM memory
4. try to define and use functions.

IMPORTS

In [1]:
import numpy as np
import polars as pl
import pandas as pd
import sys

import matplotlib.pyplot as plt
from pathlib import Path
import gc
from glob import glob
import os

import pyarrow.parquet as pq

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from sklearn.model_selection import train_test_split

# smote = SMOTE()



In [3]:
print(len(dir()))

46


FOLDER PATHS

In [4]:
ROOT            = Path("E:\\Semester-2\\DS ML\\Group-Project\\Data_Home_Credit\\")

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

#to store intermediate results, instead of keeping them in RAM
TRAIN_INTER       = ROOT / "parquet_files" / "train_inter"
TEST_INTER        = ROOT / "parquet_files" / "test_inter"

#to store intermediate results, instead of keeping them in RAM
TRAIN_INTER1       = ROOT / "parquet_files" / "train_inter1"
TEST_INTER1        = ROOT / "parquet_files" / "test_inter1"

PIPELINE CLASS - Has functions for the following use cases
1. to set the datatypes of the table
2. to handle dates --- yet to understand
3. to filter the columns --- yet to understand

In [5]:
# NOTE - The Pipeline functions here are used on the polars dataframe
class Pipeline:

    #PENDING -- Handle cases for T,L
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df): #PENDING, CHECK ALL THE DATES
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision")) #!!?
                df = df.with_columns(pl.col(col).dt.total_days().cast(pl.Int64)) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    #pass a list as argument and dont drop the columns if present in the list
    def filter_cols(df, cols_must_present=[]):
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) and (col not in cols_must_present):
                isnull = df[col].is_null().mean()
                if (isnull!= None) and (isnull > 0.7):
                    df = df.drop(col)

        #uncomment later
        # for col in df.columns:
        #     if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
        #         freq = df[col].n_unique()
        #         if (freq == 1) | (freq > 200):
        #             df = df.drop(col)
        
        return df
    


    #Group together columns based on the correlation, and select the top column from each group
    def filter_correlated_columns(df):
        #reduce the number of groups
        def reduce_group(grps):
            use = []
            for g in grps:
                mx = 0; vx = g[0]
                for gg in g:
                    n = df[gg].n_unique()
                    if n>mx:
                        mx = n
                        vx = gg
                    #print(str(gg)+'-'+str(n),', ',end='')
                use.append(vx)
                #print()
            # print('Use these',use)
            return use
        
        def group_columns_by_correlation(matrix, threshold=0.8):
            matrix=matrix.drop_nulls()
            # print('matrix',matrix)
            correlation_matrix = matrix.corr()

            # print('correlation_matrix',correlation_matrix)

            # dict to store 'columnname' as key and index as 'value'
            index_dict = {}

            # Iterate through the list and fill the dictionary
            for index, element in enumerate(matrix.columns):
                index_dict[element] = index

            # print('index dict',index_dict)

            groups = []
            remaining_cols = list(matrix.columns)
            # print('correlation matrix',correlation_matrix)

            while remaining_cols:
                col = remaining_cols.pop(0)
                group = [col]
                correlated_cols = [col]
                
                for c in remaining_cols:
                    row = index_dict[c]
                    corr_value = correlation_matrix.select(pl.col(col))
                    corr_value = corr_value[col][row]
                    # print('corr_value',corr_value)
                    if corr_value >= threshold:
                        group.append(c)
                        correlated_cols.append(c)
                groups.append(group)
                remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
            
            return groups

        #exclude string/object datatpyes - add more to the list like object Boolean
        df_changed = df.select(pl.exclude([pl.String,pl.Date]))

        #create a masked polar dataframe for df_changed
        df_nans = {}
        for col in df_changed.columns:
            df_nans[col] = df_changed[col].is_null()
        df_nans = pl.DataFrame(df_nans)
        # print(df_nans)

        #dict - key= number of missing values,, value=list having column names
        nans_groups={}
        for col in df_nans.columns:
            cur_group = df_nans[col].sum()
            try:
                nans_groups[cur_group].append(col)
            except:
                nans_groups[cur_group]=[col]

        #uses is a list which have the selected columns for the model
        uses = []
        for k,v in nans_groups.items():
            if len(v)>1:
                    # print('level 1')
                    Vs = nans_groups[k]
                    grps= group_columns_by_correlation(df[Vs], threshold=0.8)

                    # print('grps',grps)
                    use=reduce_group(grps)
                    uses=uses+use
            else:
                uses=uses+v
            print('####### NAN count =',k)
        

        #add the columns which were excluded earlier, while computing the correlation
        # Define the data types you want to select
        data_types_to_select = ['String', 'Date']  # Adjust data types as needed like Boolean, matchin the upper list
        # Get column names of selected data types
        excluded_cats = [col for col in df.columns if str(df[col].dtype) in data_types_to_select]
        uses=uses+excluded_cats

        print('columns that are not dropped after correlation are')
        print(uses)

        return df.select(uses)






    def drop_rows_with_nulls(df):
        df = df.drop_nulls()
        return df
    
    def normalize_cols(df):
   
        def normalize_column(column):
            min_val = column.min()
            max_val = column.max()
            return (column - min_val) / (max_val - min_val)
        
        # Apply Min-Max scaling to each numerical column (both Float64 and Int64)
        normalized_df = df.select([
            normalize_column(pl.col(col)).alias(col) 
            if df[col].dtype in [pl.Float64, pl.Int64] and col not in ['MONTH', 'num_group2', 'num_group1', 'date_decision', 'WEEK_NUM', 'target', 'case_id']
            else pl.col(col)
            for col in df.columns
        ])
        
        return normalized_df
    
    #Instead of encode columns use, cat_column and then pass them as categories using pandas
    # X_train[cat_cols].astype("category") --- used for pandas
    def encode_cols(df):
        for col in df.columns:
            dict1={}
            freq = df[col].n_unique()
            categories = df[col].unique().to_list()
            if freq < 200 and df[col].dtype == pl.String:
                for i in range(freq):
                    dict1[categories[i]]=i

                new_col_name = col


                df = df.with_columns( 
                        df[col].map_elements(lambda x: dict1.get(x), return_dtype=pl.UInt8).alias(new_col_name)
                )

            # print(dict1)
        return df

    #this function should fill the missing values with a valid metric
    def handle_missing_values(df):
        pass
    
                

            


    
# print(sys.getsizeof(Pipeline))

AGGREGATOR CLASS -- HAS functions to return the list of functions expressions that are passed as parameter to 'agg' function which follows 'group_by' function.
1. number expressions
2. date expressions
3. string expressions
4. Object expressions
5. group expressions(having depth=1 and depth=2)

In [6]:

class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        #expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        #expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).cast(pl.Int64).alias(f"mean_{col}") for col in cols]


        # expr_myMetric = [((pl.median(col) + pl.mean(col) + 2*pl.max(col))/4).cast(pl.Int64).alias(f"myMetric_{col}") for col in cols]

        return expr_mean #+expr_myMetric #+ expr_max #+expr_last
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        # expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        # expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]

        return  expr_max #+ expr_last +expr_mean
    
    #take max for string data types
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        # expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return  expr_max #+expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        # expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max #+expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        # expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max #+expr_last
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs
    


READ FILE/ READ FILES
1. read_file -- reads single file
2. read_files -- reads group of similar files together

In [7]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

FUNCTION TO REDUCE THE SIZE OF DATAFRAME

In [8]:
def reduce_mem_usage_polars(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.estimated_size() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = str(df[col].dtype)
        # print(col,'coltype before',col_type)
        if  col_type in  ['Int64', 'Float64']:
            #perform below operations
            c_min = df[col].min()
            c_max = df[col].max()

            if (c_max is None) or (c_min is None):
                print("heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
                continue

            if col_type == 'Int64' or col_type == 'Float64':
                # print('reduce mem if condition')
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int8))
                    # print('reduce mem if condition - Int8')
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int16))
                    # print('reduce mem if condition Int16')
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int32))
                    # print('reduce mem if condition Int32')
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int64))
                    # print('reduce mem if condition Int64')

            # elif col_type == 'Float64':
            #     if c_max < np.finfo(np.float32).max:
            #         df = df.with_columns(pl.col(col).cast(pl.Float32))
            #     else:
            #         df = df.with_columns(pl.col(col).cast(pl.Float64))
        # print(col,'coltype after',df[col].dtype)
        # print('\n')

    end_mem = df.estimated_size() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print("\n")
    
    return df

CREATE A DATA STORE FORMAT -- To perform READ/WRITE for TRAIN DATA


In [9]:
def create_data_store_train(path):
    d = {
        "df_base": [
            [read_file , path / "train_base.parquet"],
        ],

        "depth_0": [
            [read_file , path / "train_static_cb_0.parquet"],
            [read_files , path / "train_static_0_*.parquet"],
        ],

        "depth_1": [

            [read_files , path / "train_applprev_1_*.parquet", 1],
            [read_file , path / "train_tax_registry_a_1.parquet", 1],
            [read_file , path / "train_tax_registry_b_1.parquet", 1],
            [read_file , path / "train_tax_registry_c_1.parquet", 1],
            [read_files , path / "train_credit_bureau_a_1_*.parquet", 1],
            [read_file , path / "train_credit_bureau_b_1.parquet", 1],
            [read_file , path / "train_other_1.parquet", 1],
            [read_file , path / "train_person_1.parquet", 1],
            [read_file , path / "train_deposit_1.parquet", 1],
            [read_file , path / "train_debitcard_1.parquet", 1],
        ],


        "depth_2": [
            [read_file , path / "train_credit_bureau_b_2.parquet", 2],
            [read_files , path / "train_credit_bureau_a_2_*.parquet", 2],
            [read_file , path / "train_applprev_2.parquet", 2],
            [read_file , path / "train_person_2.parquet", 2],
        ]


    }


    return d

TRAINING DATA

In [10]:
data_store = create_data_store_train(TRAIN_DIR)


# print(data_store['df_base'])

print(sys.getsizeof(data_store))



184


INITIAL OPERATIONS ON THE PARQUET FILES - BOTH TRAIN AND TEST DATA
1. Read parquet files
2. Modify the dataframe accordingly
3. Store the intermediate files in seperate folder, to save RAM space
4. Below function can be used for both train/test folder

In [11]:

def updateParquetFiles(data_store,cols_must_present=[]):
    for value in data_store.values():
        for item in value:
            # print(item[1]) -- corresponds to the path of the parquet_file

            if len(item)==2:
                df = item[0](item[1]) #readfile/ readfiles happens here
            elif len(item)==3:
                df = item[0](item[1],item[2]) #readfile/ readfiles happens here
            
            #skip the empty file
            if df.shape[0] == 0:
                continue

            #filetering the colunmns of dataframe
            # 1. you can perform filter_cols here
            # 2. you can perform filter_cols in feature engineering
            df = df.pipe(Pipeline.filter_cols,cols_must_present)

            #dropping the rows with missing values -- store this result in {folder_name}_inter1
            # df = df.pipe(Pipeline.drop_rows_with_nulls)
            #INSTEAD of dropping rows having the null values 
                # 1. retain the missing values/and handle it later - This method is selected
                # 2. impute the missing values -- it is better to fill missing values after joining all tables/files

            #reduce the dataframe memory size
            df = df.pipe(reduce_mem_usage_polars)

            #WRITE df as parquet file to ../train_inter/ directory.
            # 1. Get the filename from the path
            filename = os.path.basename(item[1])
            filename = filename.replace("_*","")
            # Get the folder name from the path
            folder_name = os.path.basename(os.path.dirname(item[1]))

            # 2. Go one folder back
            parent_directory = os.path.dirname(os.path.dirname(item[1]))

            # 3. Create directory '{folder_name}_inter1' if not present
            new_directory = os.path.join(parent_directory, f"{folder_name}_inter1")
            if not os.path.exists(new_directory):
                os.makedirs(new_directory)

            # 4. Create new path
            new_path = os.path.join(new_directory, filename)

            # Write the DataFrame to a Parquet file in the new path
            df.write_parquet(new_path)




UPDATE TRAIN PARQUET FILE

In [12]:
%%time

updateParquetFiles(data_store)

Memory usage of dataframe is 52.41 MB
Memory usage after optimization is: 20.38 MB
Decreased by 61.1%


Memory usage of dataframe is 238.64 MB
Memory usage after optimization is: 117.01 MB
Decreased by 51.0%


Memory usage of dataframe is 1600.85 MB
Memory usage after optimization is: 571.50 MB
Decreased by 64.3%


Memory usage of dataframe is 261.33 MB
Memory usage after optimization is: 172.80 MB
Decreased by 33.9%


Memory usage of dataframe is 15.75 MB
Memory usage after optimization is: 9.20 MB
Decreased by 41.6%


Memory usage of dataframe is 5.19 MB
Memory usage after optimization is: 3.03 MB
Decreased by 41.6%


Memory usage of dataframe is 16.58 MB
Memory usage after optimization is: 9.68 MB
Decreased by 41.6%


Memory usage of dataframe is 707.80 MB
Memory usage after optimization is: 391.83 MB
Decreased by 44.6%


Memory usage of dataframe is 11.69 MB
Memory usage after optimization is: 6.54 MB
Decreased by 44.1%


Memory usage of dataframe is 2.73 MB
Memory usage after opti

FEATURE ENGINEERING DEFINITION

In [13]:

def feature_eng(df_base, depth_0, depth_1, depth_2):

    # print(df_base)
    folderPath = os.path.dirname(os.path.dirname(df_base[0][1]))
    folderName = os.path.basename(os.path.dirname(df_base[0][1]))

    outputFolderName = folderName.replace("inter1","inter")
    outputFolderPath = os.path.join(folderPath,outputFolderName)

    # print('outputFolderPath',outputFolderPath)


    #reading the parquet file using polars
    df_base = pl.read_parquet(df_base[0][1])

    #adding new columns to df_base
    df_base = df_base.with_columns(
        month_decision = pl.col("date_decision").dt.month().cast(pl.Int8),
        weekday_decision = pl.col("date_decision").dt.weekday().cast(pl.Int8),
    )
    # print(df_base)

    for i, list1 in enumerate(depth_0 + depth_1 + depth_2):
        filename = str(list1[1])
        
        filename = filename.replace("_*","")
        # print(filename)

        if os.path.exists(filename):
            df = pl.read_parquet(filename)
            df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")

            



    # #filter columns based on  the percentage of null values in each column - uncomment if needed
    # df_base = df_base.pipe(Pipeline.filter_cols)

    # filter_correlated_columns
    df_base = df_base.pipe(Pipeline.filter_correlated_columns)
    

    #handle rows with missing values
    # 1. drop rows with missing values
    # 2. fill the missing values using Imputers

    #dropping rows with missing values - comment later
    # df_base = df_base.pipe(Pipeline.drop_rows_with_nulls)
        
    outputFilename = os.path.join(outputFolderPath,'joined.parquet')
    print('outputfilename',outputFilename)

    #create director if not present
    if not os.path.exists(outputFolderPath):
        # Create the directory
        os.makedirs(outputFolderPath)

    df_base = df_base.pipe(Pipeline.handle_dates)

    #writing the joined.parquet file to the outputFilename
    df_base.write_parquet(outputFilename)

    
    return df_base

    # print('size of df_base',df_base)



FEATURE ENGINEERING ON TRAIN DATA

In [14]:
df_base = feature_eng(**create_data_store_train(TRAIN_INTER1))

#drop unnecessary columns that are not required for training the model
#PENDING -- pass an list as angument, dont drop the columns, if that is present in list
def drop_columns(df):
    cols_to_drop = ["case_id", "date_decision", "MONTH", "WEEK_NUM", "month_decision","weekday_decision",
                    #'dateofbirth_337D','max_birth_259D','max_contaddr_district_15M',
                    #'max_contaddr_zipcode_807M', 'max_registaddr_district_1083M', 'max_registaddr_zipcode_184M', 'max_addres_district_368M', 'max_addres_zip_823M',
                    #'max_num_group1_9', 'max_num_group1_15', 'max_num_group2_15', 'max_num_group2_14', 'max_num_group1_14', 'max_num_group2_13','max_num_group1_13'
                    ]
    df = df.drop(cols_to_drop)
    return df
df_base = drop_columns(df_base)


# df_base = df_base.pipe(Pipeline.encode_cols)

#test/submit data should have all the columns prsent in df_base.schema 
# to get the prediction from the model
# print(df_base.schema)
print('final shape of the dataframe to be given as input to ML Model',df_base.shape)

#must have columns
cols_must_present = list(df_base.schema.keys())
# print(cols_must_present)


c:\Users\kmoha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\kmoha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


####### NAN count = 0
####### NAN count = 140968
####### NAN count = 954021
####### NAN count = 418178
####### NAN count = 561124
####### NAN count = 4
####### NAN count = 613202
####### NAN count = 948244
####### NAN count = 972827
####### NAN count = 467175
####### NAN count = 624875
####### NAN count = 757006
####### NAN count = 841181
####### NAN count = 1026987
####### NAN count = 455190
####### NAN count = 460822
####### NAN count = 343375
####### NAN count = 452594
####### NAN count = 190833
####### NAN count = 442041
####### NAN count = 769046
####### NAN count = 511255
####### NAN count = 306019
####### NAN count = 960953
####### NAN count = 705504
####### NAN count = 876276
####### NAN count = 829402
####### NAN count = 1032856
####### NAN count = 766958
####### NAN count = 452593
####### NAN count = 455081
####### NAN count = 445669
####### NAN count = 456495
####### NAN count = 847191
####### NAN count = 446983
####### NAN count = 840646
####### NAN count = 669186
####### N

In [15]:
def proportion(df):
    y_counts = df['target'].value_counts()

    # Calculate the proportion of each class
    proportion_0 = y_counts[0] / len(df['target'])
    proportion_1 = y_counts[1] / len(df['target'])

    # Print the proportions
    print("Proportion of class 0:", round(proportion_0,2))
    print("Proportion of class 1:", round(proportion_1,2))
    print("\n")


MODEL TO BE TAKEN
1. Light GBM 
2. Cat Boost
3. Mix of 1 and 2

LGBM - PARAMETERS INITIALIZATIONS

In [16]:
#param initilization of light gbm

params_lgbm = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,  
    "learning_rate": 0.05,
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "device": 'cpu', 
    "verbose": -1,
}

fitted_models = []
cv_scores = []

DATA SAMPLERS INITIALIZATIONS - TO perform oversampling

In [17]:
#define stratified shuffle split

smote = SMOTE()
adasyn = ADASYN()
random_oversampler = RandomOverSampler(random_state=42)


skf = StratifiedKFold(n_splits=4, shuffle=False) #tune this parameter

TRAINING THE MODEL

Remove below section later - just for testing purpose

In [18]:
df_base_pandas = df_base.to_pandas()

print(df_base_pandas.dtypes)

print(df_base_pandas.columns)

print(df_base_pandas.select_dtypes(exclude='category').columns)


target                            int8
credamount_770A                  int32
applicationcnt_361L               int8
applications30d_658L              int8
applicationscnt_1086L            int16
                                 ...  
max_addres_district_368M        object
max_addres_zip_823M             object
max_conts_role_79M              object
max_empls_economicalst_849M     object
max_empls_employer_name_740M    object
Length: 362, dtype: object
Index(['target', 'credamount_770A', 'applicationcnt_361L',
       'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L',
       'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L',
       'clientscnt_1071L',
       ...
       'max_collaterals_typeofguarante_669M', 'max_subjectroles_name_541M',
       'max_subjectroles_name_838M', 'max_cacccardblochreas_147M',
       'max_conts_type_509L', 'max_addres_district_368M',
       'max_addres_zip_823M', 'max_conts_role_79M',
       'max_empls_economicalst_849M', 'max_e

In [19]:
cat_cols = []
num_cols = []

for key,value in df_base.schema.items():
    if str(value) in ['String','Date','Boolean']:
        cat_cols.append(key)

print('cat_cols',cat_cols)
print('len of cat_cols',len(cat_cols))

df_base = df_base.drop(cat_cols)
num_cols = df_base.columns

print('num_cols',num_cols)

cat_cols ['isbidproduct_1095L', 'opencred_647L', 'max_isbidproduct_390L', 'max_contaddr_matchlist_1032L', 'max_contaddr_smempladdr_334L', 'max_remitter_829L', 'max_safeguarantyflag_411L', 'description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M', 'requesttype_4525192L', 'credtype_322L', 'disbursementtype_67L', 'inittransactioncode_186L', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastcancelreason_561M', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastst_736L', 'paytype1st_925L', 'paytype_783L', 'previouscontdistrict_112M', 'twobodfilling_608L', 'max_cancelreason_3545846M', 'max_district_544M', 'max_education_1138M', 'max_postype_4733339M', 'max_profession_152M', 'max_rejectreason_755M', 'max_rejectreasonclient_4145042M', 'max_credtype_587L', 'max_familystate_726L', 'max_inittransactioncode_279L', 'max_status_219L', 'max_name_4527232M', 'max_name_

In [20]:

df_base_pandas = df_base.to_pandas()


df_base_pandas_y = df_base_pandas[['target']]
df_base_pandas_X = df_base_pandas.drop(['target'], axis=1)

# print(df_base_pandas_y.shape)
# print(df_base_pandas_X.shape)




proportion(df_base_pandas_y)

for train_indices, valid_indices in skf.split(df_base_pandas_X, df_base_pandas_y):
    X_train = df_base_pandas_X.iloc[train_indices]
    y_train = df_base_pandas_y.iloc[train_indices]
    
    X_valid = df_base_pandas_X.iloc[valid_indices]
    y_valid = df_base_pandas_y.iloc[valid_indices]

    print('X_train',X_train.shape)

    # #resampling using smote
    # X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    # X_valid_resampled, y_valid_resampled = smote.fit_resample(X_valid, y_valid)

    #resampling using adasyn
    # X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)
    # X_valid_resampled, y_valid_resampled = adasyn.fit_resample(X_valid, y_valid)

    #resampling using random oversampler
    # X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)
    # X_valid_resampled, y_valid_resampled = random_oversampler.fit_resample(X_valid, y_valid)

    # # without resampling
    X_train_resampled, y_train_resampled = X_train, y_train
    X_valid_resampled, y_valid_resampled = X_valid, y_valid


    
    if len(y_train_resampled.shape) > 1:
        y_train_resampled = np.ravel(y_train_resampled)

    if len(y_valid_resampled.shape) > 1:
        y_valid_resampled = np.ravel(y_valid_resampled)


    # proportion(y_train_resampled)
    # proportion(y_valid)

    # print('train_indices',len(train_indices))
    # print('valid_indices', len(valid_indices))

    #light gbm initialization
    model = lgb.LGBMClassifier(**params_lgbm)

    #training the model
    model.fit(
        X_train_resampled, y_train_resampled,
        eval_set = [(X_valid_resampled, y_valid_resampled)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)] )
    
    #adding models to the list
    fitted_models.append(model)

    y_pred_valid_resampled = model.predict_proba(X_valid_resampled)[:,1]
    auc_score = roc_auc_score(y_valid_resampled, y_pred_valid_resampled)
    cv_scores.append(auc_score)

    break #remove later



print("CV AUC scores: ", cv_scores)
print("Maximum CV AUC score: ", max(cv_scores))

    


# del df_base_pandas, df_base_pandas_X

Proportion of class 0: 0.97
Proportion of class 1: 0.03


X_train (1144994, 269)
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.778239
[400]	valid_0's auc: 0.781152
Early stopping, best iteration is:
[368]	valid_0's auc: 0.781559
CV AUC scores:  [0.7815589164437025]
Maximum CV AUC score:  0.7815589164437025


In [21]:
fitted_models

[LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='cpu',
                extra_trees=True, learning_rate=0.05, max_depth=10, metric='auc',
                n_estimators=2000, num_leaves=64, objective='binary',
                random_state=42, reg_alpha=0.1, reg_lambda=10, verbose=-1)]

STORE THE MODEL IN A GIVEN PATH

In [22]:
#stroing the trained model

from joblib import dump

print(TRAIN_INTER)

dump(fitted_models, f'{TRAIN_INTER}/lgbm_model_.joblib')

# for i, model in enumerate(fitted_models):
#     dump(model, f'{TRAIN_INTER}/model_{i}.joblib')

E:\Semester-2\DS ML\Group-Project\Data_Home_Credit\parquet_files\train_inter


['E:\\Semester-2\\DS ML\\Group-Project\\Data_Home_Credit\\parquet_files\\train_inter/lgbm_model_.joblib']

LOAD THE MODEL

In [23]:
import joblib

# Load the model
model = joblib.load(TRAIN_INTER / 'lgbm_model_.joblib')

# print(model)
# Now you can use the loaded model for predictions or other tasks

cols_used_for_training = []

for i in model:
    
    # Print type of model
    print("Model type:", type(i))

    # Print model attributes and methods
    # print("Model attributes and methods:", dir(i))

    
    # get the list of feature names on which the model is trained
    print("feature_name_:", i.feature_name_)

    cols_used_for_training= i.feature_name_


    print(len(i.feature_name_))

        
    break

fitted_models = model


Model type: <class 'lightgbm.sklearn.LGBMClassifier'>
feature_name_: ['credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L', 'mean_mainoccupationinc_384A', 'max_num_group1_9', 'days180_256L', 'days30_165L', 'days360_512L', 'firstquarter_103L', 'fourthquarter_440L', 'secondquarter_766L', 'thirdquarter_1082L', 'mean_debtoutstand_525A', 'mean_debtoverdue_47A', 'pmtscount_423L', 

VOTING MODEL

In [24]:
from scipy.stats import mode

class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        mode_result = mode(y_preds).mode
        return y_preds[0]
     
    def predict_proba(self, X):      
        # lgb
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        # # cat        
        # X[cat_cols] = X[cat_cols].astype(str)
        # y_preds += [estimator.predict_proba(X) for estimator in self.estimators[-5:]]
        
        return np.mean(y_preds, axis=0)
    
model = VotingModel(fitted_models)

print(len(model.estimators))

1


READ THE PARQUET FILE (TRAINING)

In [25]:
#read the parquet file
filepath = TRAIN_INTER / "joined.parquet"
df_base = pl.read_parquet(filepath)

# df_base = df_base.pipe(Pipeline.encode_cols)


SPLIT THE DATA USING SHUFFLE SPLIT TO TEST THE GINI STABILITY

In [26]:
def getPandasdata_forIds(df, case_ids):
    df = df.filter(pl.col("case_id").is_in(case_ids))
    return (
        df.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", 'target']].to_pandas(),
        df[cols_used_for_training].select(pl.exclude(['target'])).to_pandas(),
        df[['target']].to_pandas()
    )



#convert series data to numpy array, in order to send to test_train_split
case_ids = df_base["case_id"].unique().shuffle(seed=1).to_numpy()


case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.8, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.6, random_state=1)

#test the data by following the starter notebook
base_train, X_train, y_train = getPandasdata_forIds(df_base, case_ids_train)
base_valid, X_valid, y_valid = getPandasdata_forIds(df_base, case_ids_valid)
base_test, X_test, y_test = getPandasdata_forIds(df_base, case_ids_test)

print('Shape of X_train',X_train.shape)
print('Shape of X_valid',X_valid.shape)
print('Shape of X_test',X_test.shape)

print(X_train.columns)



Shape of X_train (1221327, 269)
Shape of X_valid (183199, 269)
Shape of X_test (122133, 269)
Index(['credamount_770A', 'applicationcnt_361L', 'applications30d_658L',
       'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L',
       'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L',
       'clientscnt_1130L',
       ...
       'max_refreshdate_3813885D', 'max_contractdate_551D',
       'max_contractmaturitydate_151D', 'max_lastupdate_260D',
       'max_birth_259D', 'max_empl_employedfrom_271D',
       'max_contractenddate_991D', 'max_openingdate_313D',
       'max_openingdate_857D', 'max_pmts_date_1107D'],
      dtype='object', length=269)


In [27]:
print(set(num_cols)-set(X_train.columns))

{'target'}


PREDICTION FROM THE MODEL

In [28]:
# print(X_train.dtypes)
s = set()
l = []
for key,value in X_train.dtypes.items():
    s.add(value)
    if value in ['datetime64[ms]']:
        l.append(key)

print(s)
print(l)

{dtype('float64'), dtype('int8'), dtype('int64'), dtype('int32'), dtype('int16')}
[]


In [29]:

for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = model.predict_proba(X)[:,1]
    base["score"] = y_pred

In [30]:
print('base.test',base_test.columns)
print('y_test',y_test.columns)

base.test Index(['case_id', 'WEEK_NUM', 'target', 'score'], dtype='object')
y_test Index(['target'], dtype='object')


ROC AND AUC CURVE FOR THE DATA

In [31]:
print(f'The AUC score on the train set is: {roc_auc_score(y_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(y_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(y_test["target"], base_test["score"])}')  

The AUC score on the train set is: 0.8607784409786201
The AUC score on the valid set is: 0.8572637854566031
The AUC score on the test set is: 0.8584569327608836


In [32]:
print(base_train['target'].value_counts())
print(base_valid['target'].value_counts())
print(base_test['target'].value_counts())

target
0    1182976
1      38351
Name: count, dtype: int64
target
0    177496
1      5703
Name: count, dtype: int64
target
0    118193
1      3940
Name: count, dtype: int64


GINI SCORE DEFINITION

In [33]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
    .sort_values("WEEK_NUM")\
    .groupby("WEEK_NUM")[["target", "score"]]\
    .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1 if len(x["target"].unique()) > 1 else 0).tolist()
    

    # print(base)

    # for group_name, group_data in updated_base:
    #     print(f"Group: {group_name}")
    #     print(group_data)
    #     print() 

    # print(base['target'].value_counts())

    gini_in_time = [x for x in gini_in_time if x > 0]

    # print(gini_in_time)
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

COMPUTE THE GINI SCORES FOR THE DATA

In [34]:
stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}') 

The stability score on the train set is: 0.7024239236847906
The stability score on the valid set is: 0.6826155517694663
The stability score on the test set is: 0.6820645269280128


=================== TEST DATA PROCESSING STARTS FROM HERE =================

CREATE DATA STORE FOR TEST DATA

In [35]:
def create_data_store_test(path):
    d = {
        "df_base": [
            [read_file , path / "test_base.parquet"],
        ],

        "depth_0": [
            [read_file , path / "test_static_cb_0.parquet"],
            [read_files , path / "test_static_0_*.parquet"],
        ],

        "depth_1": [

            [read_files , path / "test_applprev_1_*.parquet", 1],
            [read_file , path / "test_tax_registry_a_1.parquet", 1],
            [read_file , path / "test_tax_registry_b_1.parquet", 1],
            [read_file , path / "test_tax_registry_c_1.parquet", 1],
            [read_files , path / "test_credit_bureau_a_1_*.parquet", 1],
            [read_file , path / "test_credit_bureau_b_1.parquet", 1],
            [read_file , path / "test_other_1.parquet", 1],
            [read_file , path / "test_person_1.parquet", 1],
            [read_file , path / "test_deposit_1.parquet", 1],
            [read_file , path / "test_debitcard_1.parquet", 1],
        ],


        "depth_2": [
            [read_file , path / "test_credit_bureau_b_2.parquet", 2],
            [read_files , path / "test_credit_bureau_a_2_*.parquet", 2],
            [read_file , path / "test_applprev_2.parquet", 2],
            [read_file , path / "test_person_2.parquet", 2],
        ]


    }


    return d


In [36]:
#creating the data store format for the given test data 
data_store = create_data_store_test(TEST_DIR)

UPDATEPARQUETFILES DEFINITION FOR TEST DATA

In [37]:

def updateParquetFiles_test(data_store,cols_must_present=[]):
    for value in data_store.values():
        for item in value:
            # print(item[1]) -- corresponds to the path of the parquet_file

            if len(item)==2:
                df = item[0](item[1]) #readfile/ readfiles happens here
            elif len(item)==3:
                df = item[0](item[1],item[2]) #readfile/ readfiles happens here
            
            #skip the empty file
            if df.shape[0] == 0:
                continue

            #reduce the dataframe memory size
            df = df.pipe(reduce_mem_usage_polars)

            #WRITE df as parquet file to ../train_inter/ directory.
            # 1. Get the filename from the path
            filename = os.path.basename(item[1])
            filename = filename.replace("_*","")
            # # Get the folder name from the path
            folder_name = os.path.basename(os.path.dirname(item[1]))

            # print(folder_name)

            # 2. Go one folder back
            parent_directory = os.path.dirname(os.path.dirname(item[1]))

            # 3. Create directory '{folder_name}_inter1' if not present
            new_directory = os.path.join(parent_directory, f"{folder_name}_inter1")

            

            if not os.path.exists(new_directory):
                os.makedirs(new_directory)

            # 4. Create new path
            new_path = os.path.join(new_directory, filename)

            # Write the DataFrame to a Parquet file in the new path
            df.write_parquet(new_path)

UPDATE TEST PARQUET FILES TO TEST1 FOLDER

In [38]:
updateParquetFiles_test(data_store)

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 53.6%


Memory usage of dataframe is 0.00 MB
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
heyyyyyyyyyyyyyy

FEATURE ENGINEERING DEFINTION FOR TEST DATA

In [39]:

def feature_eng_test(df_base, depth_0, depth_1, depth_2):

    # print(df_base)
    folderPath = os.path.dirname(os.path.dirname(df_base[0][1]))
    folderName = os.path.basename(os.path.dirname(df_base[0][1]))

    outputFolderName = folderName.replace("inter1","inter")
    outputFolderPath = os.path.join(folderPath,outputFolderName)

    # print('outputFolderPath',outputFolderPath)


    #reading the parquet file using polars
    df_base = pl.read_parquet(df_base[0][1])

    #adding new columns to df_base
    df_base = df_base.with_columns(
        month_decision = pl.col("date_decision").dt.month().cast(pl.Int8),
        weekday_decision = pl.col("date_decision").dt.weekday().cast(pl.Int8),
    )
    # print(df_base)

    for i, list1 in enumerate(depth_0 + depth_1 + depth_2):
        filename = str(list1[1])
        
        filename = filename.replace("_*","")
        # print(filename)

        if os.path.exists(filename):
            df = pl.read_parquet(filename)
            df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")

            
        
    outputFilename = os.path.join(outputFolderPath,'joined.parquet')
    print('outputfilename',outputFilename)

    #create director if not present
    if not os.path.exists(outputFolderPath):
        # Create the directory
        os.makedirs(outputFolderPath)

    df_base = df_base.pipe(Pipeline.handle_dates)

    #writing the joined.parquet file to the outputFilename
    df_base.write_parquet(outputFilename)

    
    return df_base

    # print('size of df_base',df_base)



FEATURE ENGINEERING ON TEST DATA

In [40]:
df_base_submission = feature_eng_test(**create_data_store_test(TEST_INTER1))

#keep columns in 'df' that are present while training the model
#PENDING -- pass an list as angument, dont drop the columns, if that is present in list
def keep_columns(df,cols_must_present=[]):
    for col in df.columns:
        if col not in cols_must_present:
            df = df.drop(col)
    
    return df

# df_base_submission = keep_columns(df_base_submission,cols_must_present)


# df_base_submission = df_base_submission.pipe(Pipeline.encode_cols)

print(df_base_submission)

# a = set(cols_must_present)
# b= set(df_base_submission.schema.keys())

# print(a-b)


# for key,val in df_base.schema.items():
    
#     if str(val) in ['String']:
#         print(key)

outputfilename E:\Semester-2\DS ML\Group-Project\Data_Home_Credit\parquet_files\test_inter\joined.parquet
shape: (10, 482)
┌─────────┬──────────┬────────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ case_id ┆ WEEK_NUM ┆ month_deci ┆ weekday_de ┆ … ┆ max_addre ┆ max_relat ┆ max_num_g ┆ max_num_g │
│ ---     ┆ ---      ┆ sion       ┆ cision     ┆   ┆ s_role_87 ┆ edpersons ┆ roup1_15  ┆ roup2_15  │
│ i32     ┆ i8       ┆ ---        ┆ ---        ┆   ┆ 1L        ┆ _role_762 ┆ ---       ┆ ---       │
│         ┆          ┆ i8         ┆ i8         ┆   ┆ ---       ┆ T         ┆ i8        ┆ i8        │
│         ┆          ┆            ┆            ┆   ┆ str       ┆ ---       ┆           ┆           │
│         ┆          ┆            ┆            ┆   ┆           ┆ str       ┆           ┆           │
╞═════════╪══════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 57543   ┆ 100      ┆ 5          ┆ 5          ┆ … ┆ null      ┆ null

In [63]:
# print(df_base_submission.dtypes)
for index, col in enumerate(df_base_submission.columns):
    # print(index, df_base_submission[col].dtype)
    pass
        
        
l1 = [48,52, 414,415, 469]

l = df_base_submission.columns

for i in l1:
    print(l[1])


WEEK_NUM
WEEK_NUM
WEEK_NUM
WEEK_NUM
WEEK_NUM


In [41]:
# selected_columns = num_cols

# selected_columns = cols_used_for_training

if 'target' in cols_used_for_training:
    cols_used_for_training.remove('target')

print(len(cols_used_for_training))

print('missed columns',set(cols_used_for_training)-set(df_base_submission.columns))

missed_cols = set(cols_used_for_training)-set(df_base_submission.columns)
final_cols = list(set(cols_used_for_training)-set(missed_cols)) + ['case_id']

df_base_submission_changed = df_base_submission.select(final_cols)

# print(df_base_submission_changed)

print('num of cols before',len(df_base_submission_changed.columns))

for i in list(missed_cols):
    df_base_submission_changed = df_base_submission_changed.with_columns(
        pl.lit(None).cast(pl.Int8).alias(i)
    )

print('num of cols after',len(df_base_submission_changed.columns))

# print(df_base_submission_changed[['mean_pmtamount_36A']])



269
missed columns {'max_num_group1_5', 'mean_pmtamount_36A', 'max_processingdate_168D'}
num of cols before 267
num of cols after 270


OUTPUT PREDICTION FOR TEST DATA

In [42]:
#predict the output score

df_base_submission_pandas = df_base_submission_changed.to_pandas()
df_base_submission_pandas = df_base_submission_pandas.set_index("case_id")

y_pred = pd.Series(model.predict_proba(df_base_submission_pandas)[:, 1], index=df_base_submission_pandas.index)

df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred
df_subm.to_csv(ROOT / "submission_result.csv")

print(df_subm)

# print(df_base_submission_pandas)


            score
case_id          
57543    0.187879
57549    0.155747
57551    0.144672
57552    0.124722
57569    0.302953
57630    0.136577
57631    0.127497
57632    0.186687
57633    0.115593
57634    0.150649
